In [1]:
import glob
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import csv

pd.options.display.float_format = '{:.5f}'.format

In [2]:
use_counties = ['台北市', '高雄市', '新北市', '桃園市', '台中市', '台南市', '苗栗縣', '新竹縣', '基隆市',
       '屏東縣', '新竹市', '宜蘭縣', '花蓮縣', '嘉義市', '金門縣', '嘉義縣', '彰化縣', '雲林縣']
dfs = []
for pathto_file in glob.glob('../外部資料集/實價登錄/*/*_a.csv'):
    df = pd.read_csv(pathto_file, on_bad_lines='skip', skiprows=[1])
    df['單價元平方公尺'].fillna(0)
    df['土地位置建物門牌'] = df['土地位置建物門牌'].apply(lambda x: (x.replace('臺北市', '台北市').replace('臺中市', '台中市').replace('臺南市', '台南市')))
    county = df['土地位置建物門牌'].apply(lambda x: (x+'123')[:3]).value_counts().index[0]
    if county not in use_counties:
        continue
    df['縣市'] = county
    avg_price = df.query('單價元平方公尺 > 1000')['單價元平方公尺'].median()
    avg_price = avg_price*0.1
    df = df.query(f'單價元平方公尺 > {avg_price}')
    df['source'] = pathto_file.split('/')[-2]
    dfs.append(df)
df = pd.concat(dfs).reset_index(drop=True)

/tmp/ipykernel_21033/16003613.py:5: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(pathto_file, on_bad_lines='skip', skiprows=[1])
/tmp/ipykernel_21033/16003613.py:5: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(pathto_file, on_bad_lines='skip', skiprows=[1])


In [3]:
use_sources = ['2021Q1', '2021Q2', '2021Q3', '2021Q4', '2022Q1', '2022Q2', '2022Q3', '2022Q4', '2023Q1', '2023Q2', '2023Q3']

print(df.shape)
df = df[df['source'].isin(use_sources)].copy().reset_index(drop=True)
print(df.shape)

(1000242, 35)
(740604, 35)


In [4]:
df.query('交易年月日 >= 1091000 and 交易年月日 <= 1091231').shape

(18600, 35)

In [5]:
df.query('交易年月日 >= 1090700 and 交易年月日 <= 1090931').shape

(17898, 35)

In [6]:
set(use_counties) - set(df['縣市'].unique().tolist())

set()

In [7]:
df['縣市'].value_counts()

縣市
新北市    125504
台中市    113971
桃園市    103404
高雄市     93255
台南市     68294
台北市     49944
彰化縣     27836
新竹縣     27028
屏東縣     22506
苗栗縣     17610
宜蘭縣     17413
新竹市     17374
雲林縣     14191
基隆市     11886
嘉義縣     11608
花蓮縣      8729
嘉義市      7907
金門縣      2144
Name: count, dtype: int64

In [8]:
df.groupby('鄉鎮市區')['單價元平方公尺'].mean()

鄉鎮市區
七堵區    58568.32041
七股區    32477.03030
三地門鄉   14841.45455
三峽區    84851.30799
三星鄉    44033.19048
           ...    
鼓山區    86422.33455
龍井區    54763.70485
龍崎區    19526.83333
龍潭區    51587.97717
龜山區    79077.76022
Name: 單價元平方公尺, Length: 318, dtype: float64

In [9]:
df = df.query('主建物面積 >= 2').copy().reset_index(drop=True)
df = df.query('主建物面積 <= 2000').copy().reset_index(drop=True)
df = df.query('陽台面積 <= 100').copy().reset_index(drop=True)
df = df.query('單價元平方公尺 >= 1000')
df = df.query('單價元平方公尺 <= 2000000').reset_index(drop=True)

col = '主建物面積'
scaler = StandardScaler()
scaler.fit(df[col].apply(np.sqrt).to_numpy().reshape(-1, 1))
df[col] = scaler.transform(df[col].apply(np.sqrt).to_numpy().reshape(-1, 1))
print(df[col].describe())

col = '陽台面積'
scaler = StandardScaler()
scaler.fit(df[col].apply(np.sqrt).to_numpy().reshape(-1, 1))
df[col] = scaler.transform(df[col].apply(np.sqrt).to_numpy().reshape(-1, 1))
print(df[col].describe())

col = '附屬建物面積'
scaler = StandardScaler()
scaler.fit(df[col].apply(np.sqrt).to_numpy().reshape(-1, 1))
df[col] = scaler.transform(df[col].apply(np.sqrt).to_numpy().reshape(-1, 1))
print(df[col].describe())

col = '車位面積'
df[col] = df['車位移轉總面積平方公尺'].values
scaler = StandardScaler()
scaler.fit(df[col].apply(np.sqrt).to_numpy().reshape(-1, 1))
df[col] = scaler.transform(df[col].apply(np.sqrt).to_numpy().reshape(-1, 1))
print(df[col].describe())

col = '建物面積'
df[col] = df['建物移轉總面積平方公尺'].values
scaler = StandardScaler()
scaler.fit(df[col].apply(np.sqrt).to_numpy().reshape(-1, 1))
df[col] = scaler.transform(df[col].apply(np.sqrt).to_numpy().reshape(-1, 1))
print(df[col].describe())

col = '土地面積'
df[col] = df['土地移轉總面積平方公尺'].values
scaler = StandardScaler()
scaler.fit(df[col].apply(np.sqrt).to_numpy().reshape(-1, 1))
df[col] = scaler.transform(df[col].apply(np.sqrt).to_numpy().reshape(-1, 1))
print(df[col].describe())

# col = '單價'
# df[col] = df['單價元平方公尺'].values
# scaler = StandardScaler()
# scaler.fit(df[col].apply(np.sqrt).to_numpy().reshape(-1, 1))
# df[col] = scaler.transform(df[col].apply(np.sqrt).to_numpy().reshape(-1, 1)) + 2
# print(df[col].describe())

count   597111.00000
mean        -0.00000
std          1.00000
min         -2.55662
25%         -0.61966
50%         -0.19512
75%          0.39355
max         11.26292
Name: 主建物面積, dtype: float64
count   597111.00000
mean         0.00000
std          1.00000
min         -1.70557
25%         -0.39962
50%          0.13465
75%          0.59945
max          5.29039
Name: 陽台面積, dtype: float64
count   597111.00000
mean         0.00000
std          1.00000
min         -0.59497
25%         -0.59497
50%         -0.59497
75%          0.53850
max         21.28760
Name: 附屬建物面積, dtype: float64
count   597111.00000
mean         0.00000
std          1.00000
min         -0.71671
25%         -0.71671
50%         -0.71671
75%          1.07916
max         15.22237
Name: 車位面積, dtype: float64
count   597111.00000
mean        -0.00000
std          1.00000
min         -3.11599
25%         -0.59409
50%         -0.07020
75%          0.46983
max         17.28928
Name: 建物面積, dtype: float64
count   597111.00000
m

In [10]:
def get_year_from_str(x):
    try:
        x = int(x)
        if x < 1000101:
            return int(str(x)[0:2])
        else:
            return int(str(x)[0:3])
    except:
        return -1

def get_month_from_str(x):
    try:
        x = int(x)
        if x < 1000101:
            return int(str(x)[2:4])
        else:
            return int(str(x)[3:5])
    except:
        return -1

In [11]:
df['Trade_Year'] = df['交易年月日'].apply(get_year_from_str).astype(int) + 1911
df['Trade_Month'] = df['交易年月日'].apply(get_month_from_str).astype(int)
df['Trade_YearMonth'] = df['Trade_Year']*100 + df['Trade_Month']

In [12]:
df = df.query('Trade_Month > 0').copy().reset_index(drop=True)

In [13]:
df['Trade_Q'] = df['Trade_Month'].apply(lambda x : (x-1) // 3)
df = df.query('Trade_Q >= 0 and Trade_Q <= 3').copy().reset_index(drop=True)
df['Trade_Q'].value_counts()

Trade_Q
1    159620
2    156999
3    151921
0    128566
Name: count, dtype: int64

In [14]:
df['Trade_YearQ'] = df['Trade_Year']*10 + df['Trade_Q']
df['Trade_YearQ'].value_counts()

Trade_YearQ
20213    73828
20212    66692
20220    55691
20221    50041
20231    45672
         ...  
19883        1
20063        1
19952        1
19811        1
20080        1
Name: count, Length: 66, dtype: int64

In [15]:
df = df[~df['建築完成年月'].isna()].copy().reset_index(drop=True)

In [16]:
df['Build_Year'] = df['建築完成年月'].apply(get_year_from_str).astype(int) + 1911
df['Build_Month'] = df['建築完成年月'].apply(get_month_from_str).astype(int)

In [17]:
df = df.query('Build_Year > 1910')
df = df.query('Build_Month > 0 and Build_Month <= 12').copy().reset_index(drop=True)

In [18]:
pd.set_option('display.max_columns', None)
df = df.query('Trade_Year >= 2021 and Trade_Year <= 2022')

In [19]:
df['屋齡'] = df['Trade_Year'] - df['Build_Year'] + (df['Trade_Month'] - df['Build_Month'])/12
df = df.query('屋齡 >= 0 and 屋齡 <= 60.1').copy().reset_index(drop=True)
df['屋齡'].describe()

count   329326.00000
mean        20.99939
std         15.42516
min          0.00000
25%          5.83333
50%         23.16667
75%         32.16667
max         60.08333
Name: 屋齡, dtype: float64

In [20]:
df['交易標的'].value_counts()

交易標的
房地(土地+建物)       205689
房地(土地+建物)+車位    121925
建物                1712
Name: count, dtype: int64

In [21]:
df['建物型態'].value_counts()

建物型態
住宅大樓(11層含以上有電梯)    134976
透天厝                 85497
華廈(10層含以下有電梯)       62331
公寓(5樓含以下無電梯)        45192
套房(1房1廳1衛)           1091
店面(店鋪)                136
廠辦                     70
其他                     27
辦公商業大樓                  6
Name: count, dtype: int64

In [22]:
df = df[~df['總樓層數'].isna()].copy()
df = df[~df['總樓層數'].isna()].copy()
df = df.query('交易標的 != "土地"').copy()
df = df.reset_index(drop=True)
df['交易標的'].value_counts()

交易標的
房地(土地+建物)       205388
房地(土地+建物)+車位    121891
建物                1711
Name: count, dtype: int64

In [23]:
df['單價元平方公尺'].apply(lambda x: x / 60000) .describe()

count   328990.00000
mean         1.49524
std          1.08474
min          0.03172
25%          0.86540
50%          1.21362
75%          1.78277
max         32.40138
Name: 單價元平方公尺, dtype: float64

In [24]:
floor_dict = {
    '二十': 20,
    '三十': 30,
    '四十': 40,
    '五十': 50,
    '六十': 60,
    '七十': 70,
    '八十': 80,
    '九十': 90,
}
floor_dict2 = {
    '一': 1,
    '二': 2,
    '三': 3,
    '四': 4,
    '五': 5,
    '六': 6,
    '七': 7,
    '八': 8,
    '九': 9,
}
def get_floor(x):
    x = str(x)
    floor = 0
    for k, v in floor_dict.items():
        if x.startswith(k):
            x = x.replace(k, '')
            floor += v
    if x.startswith('十'):
        x = x.replace('十', '')
        floor += 10
    for k, v in floor_dict2.items():
        if x.startswith(k):
            x = x.replace(k, '')
            floor += v
    return floor

df['total_floor'] = df['總樓層數'].apply(get_floor)
df['floor'] = df['移轉層次'].apply(get_floor)

In [25]:
def get_road_name(row):
    road_name = row['土地位置建物門牌']
    if road_name.startswith(row['縣市']):
        road_name = road_name.replace(row['縣市'], '')
    if road_name.startswith(row['鄉鎮市區']):
        road_name = road_name.replace(row['鄉鎮市區'], '')
    if '段' in road_name:
        road_name = road_name.split('段')[0] + '段'
    elif '路' in road_name:
        road_name = road_name.split('路')[0] + '路'
    elif '街' in road_name:
        road_name = road_name.split('街')[0] + '街'
    else:
        road_name = ""
    return road_name
    
df['路名'] = df.apply(lambda row: get_road_name(row), axis=1)

In [26]:
df = df.query('floor > 0').reset_index(drop=True)
df = df.query('total_floor > 0').reset_index(drop=True)
df = df.query('total_floor >= floor').reset_index(drop=True)
df = df.query('路名 != ""').reset_index(drop=True)

In [27]:
df.sample(5)

,鄉鎮市區,交易標的,土地位置建物門牌,土地移轉總面積平方公尺,都市土地使用分區,非都市土地使用分區,非都市土地使用編定,交易年月日,交易筆棟數,移轉層次,總樓層數,建物型態,主要用途,主要建材,建築完成年月,建物移轉總面積平方公尺,建物現況格局-房,建物現況格局-廳,建物現況格局-衛,建物現況格局-隔間,有無管理組織,總價元,單價元平方公尺,車位類別,車位移轉總面積平方公尺,車位總價元,備註,編號,主建物面積,附屬建物面積,陽台面積,電梯,移轉編號,縣市,source,車位面積,建物面積,土地面積,Trade_Year,Trade_Month,Trade_YearMonth,Trade_Q,Trade_YearQ,Build_Year,Build_Month,屋齡,total_floor,floor,路名
204016,中和區,房地(土地+建物),新北市中和區景平路６０１號七樓之二十九,8.39000,工,NaN,NaN,1111011,土地1建物1車位0,七層,十一層,住宅大樓(11層含以上有電梯),辦公用,鋼筋混凝土造,921125.00000,63.09000,0,0,0,無,有,10500000,166429.00000,NaN,0.00000,0,NaN,RPPQMLLJJIHGFHF87DA,-1.05377,-0.59497,-0.43976,有,NaN,新北市,2022Q4,-0.71671,-1.12548,-0.68277,2022,10,202210,3,20223,2003,11,18.91667,11,7,景平路
32210,新竹市,房地(土地+建物)+車位,新竹市新竹市中正路１９８巷２３號六樓之５,22.43000,都市：其他:第二種住宅區,NaN,NaN,1100520,土地3建物1車位1,六層,十層,華廈(10層含以下有電梯),住家用,鋼筋混凝土造,1100205.00000,112.03000,2,2,1,有,無,10000000,105854.00000,坡道平面,33.62000,1700000,預售屋、或土地及建物分件登記案件；,RPROMLSJOHGGFAO09DA,-0.75400,-0.59497,-0.48523,有,NaN,新竹市,2021Q3,1.36090,-0.31494,-0.28384,2021,5,202105,1,20211,2021,2,0.25000,10,6,中正路
127846,安平區,房地(土地+建物)+車位,台南市安平區府平路３８８號十三樓之６,21.08000,住,NaN,NaN,1110410,土地1建物1車位1,十三層,十五層,住宅大樓(11層含以上有電梯),住家用,鋼筋混凝土構造,1100930.00000,166.30000,3,2,2,有,有,12700000,76368.00000,坡道平面,37.66000,0,含傢俱設備費；,RPROMLQKNHHGFAD87DA,-0.20419,-0.59497,0.32390,有,522.00000,台南市,2022Q2,1.48219,0.39429,-0.31523,2022,4,202204,1,20221,2021,9,0.58333,15,13,府平路
9131,信義區,房地(土地+建物),基隆市信義區崇法街８９巷２號七樓,30.29000,住,NaN,NaN,1100517,土地1建物1車位0,七層,九層,華廈(10層含以下有電梯),住家用,鋼筋混凝土造,1090513.00000,100.72000,2,2,1,有,有,6100000,60564.00000,NaN,0.00000,0,NaN,RPOPMLLJPHGGFDC37DA,-0.61519,0.94904,0.36348,有,NaN,基隆市,2021Q3,-0.71671,-0.48324,-0.11736,2021,5,202105,1,20211,2020,5,1.00000,9,7,崇法街
162976,北屯區,房地(土地+建物)+車位,台中市北屯區環太東路６２３號十二樓之６,23.23000,住,NaN,NaN,1110507,土地1建物1車位1,十二層,十四層,住宅大樓(11層含以上有電梯),住家用,鋼筋混凝土造,1040615.00000,138.77000,3,2,2,有,有,12500000,90077.00000,坡道平面,21.58000,0,NaN,RPSOMLTJOHHGFBB28EA,-0.31732,-0.59497,0.46775,有,1809.00000,台中市,2022Q3,0.94782,0.05194,-0.26568,2022,5,202205,1,20221,2015,6,6.91667,14,12,環太東路


In [28]:
df['總樓層數'] = df['total_floor']
df['移轉層次'] = df['floor']

In [29]:
print(df.shape)
df = df.query('移轉層次 >= 2 and 移轉層次 <= 46')
print(df.shape)
df = df.query('總樓層數 >= 2 and 總樓層數 <= 68')
print(df.shape)

(239981, 49)
(220087, 49)
(219989, 49)


In [30]:
def get_car_cnt(x):
    x = x.split('車位')[-1]
    if x.isdigit():
        return int(x)
    return -999999

In [31]:
df['車位個數'] = df['交易筆棟數'].apply(get_car_cnt).astype(int)
df['車位個數'].describe()

count   219989.00000
mean         0.60487
std          0.75483
min          0.00000
25%          0.00000
50%          1.00000
75%          1.00000
max         31.00000
Name: 車位個數, dtype: float64

In [32]:
df['單價'] = df['單價元平方公尺'].apply(lambda x: x/58000)
df['單價'].describe()

count   219989.00000
mean         1.63138
std          1.05441
min          0.07217
25%          0.95081
50%          1.33412
75%          1.99172
max         15.59512
Name: 單價, dtype: float64

In [33]:
df = df.query('單價 > 0').copy().reset_index(drop=True)

In [34]:
df['格局_房'] = df['建物現況格局-房']
df['格局_廳'] = df['建物現況格局-廳']
df['格局_衛'] = df['建物現況格局-衛']
df['格局_隔間'] = df['建物現況格局-隔間'].map({'有': True, '無': False})
#df['格局_電梯'] = df['電梯'].map({'有': True, '無': False})
df['格局_管理'] = df['有無管理組織'].map({'有': True, '無': False})

In [35]:
dfs_build = []
for pathto_file in glob.glob('../外部資料集/實價登錄/*/*_a_build.csv'):
    _df = pd.read_csv(pathto_file, skiprows=[1])
    dfs_build.append(_df)
df_build = pd.concat(dfs_build).reset_index(drop=True)

useages = ['住家用', '集合住宅', '其他', '店鋪', '商業用', '國民住宅', '住工用', '一般事務所', '住商用',
       '廠房', '工業用', '辦公室']
print(df_build.shape)
print(set(useages) - set(df_build.主要用途.unique().tolist()))
df_build = df_build[df_build['主要用途'].isin(useages)]
print(df_build.shape)
df_build = df_build.drop_duplicates(subset=['編號'], keep='first')
print(df_build.shape)

df_build_usage_map = dict(zip(df_build['編號'].values, df_build['主要用途'].values))

df['主要用途_from_info'] = df['編號'].map(df_build_usage_map)
df = df[~df['主要用途_from_info'].isna()].copy().reset_index(drop=True)
df = df[~df['主要用途'].isna()].copy().reset_index(drop=True)
print(df['主要用途'].value_counts())
print(df['主要用途_from_info'].value_counts())
df['主要用途'] = df['主要用途_from_info']

(2415332, 9)
set()
(962995, 9)
(894637, 9)
主要用途
住家用    196598
辦公用      5112
商業用      5005
工業用      1692
住商用       356
其他         46
住工用        41
工商用        28
Name: count, dtype: int64
主要用途_from_info
住家用      133156
集合住宅      61049
商業用        5806
辦公室        4010
一般事務所      2577
工業用        1204
廠房          519
住商用         411
國民住宅         59
店鋪           44
住工用          43
Name: count, dtype: int64


In [36]:
use_cols = ['縣市', '鄉鎮市區', '路名', '建物型態', '主要用途', 'Trade_YearQ', '單價']

df[use_cols].to_csv('../外部資料集/實價登錄/external_gov_data_by_year.csv', index=False)
df[use_cols].head(10)

,縣市,鄉鎮市區,路名,建物型態,主要用途,Trade_YearQ,單價
0,台北市,中山區,新生北路二段,住宅大樓(11層含以上有電梯),商業用,20212,2.92557
1,台北市,士林區,格致路,華廈(10層含以下有電梯),商業用,20212,2.50705
2,台北市,士林區,延平北路六段,公寓(5樓含以下無電梯),住家用,20212,1.88367
3,台北市,中山區,植福路,住宅大樓(11層含以上有電梯),住家用,20212,11.63088
4,台北市,信義區,忠孝東路五段,華廈(10層含以下有電梯),住家用,20212,5.00043
5,台北市,松山區,南京東路五段,住宅大樓(11層含以上有電梯),住家用,20212,4.17467
6,台北市,內湖區,成功路四段,住宅大樓(11層含以上有電梯),一般事務所,20212,4.23005
7,台北市,信義區,信義路五段,住宅大樓(11層含以上有電梯),住家用,20212,4.14955
8,台北市,中山區,新生北路三段,華廈(10層含以下有電梯),住家用,20212,2.46779
9,台北市,大安區,市民大道三段,住宅大樓(11層含以上有電梯),住家用,20212,4.76069


In [37]:
df.Trade_YearQ.value_counts()

Trade_YearQ
20213    40799
20212    36395
20220    31742
20221    28923
20222    26665
20223    25127
20211    17390
20210     1837
Name: count, dtype: int64